In [3]:
!pip install scikit-surprise --quiet


In [4]:
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -q ml-100k.zip


--2025-07-01 16:18:13--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip.2’

ml-100k.zip.2       100%[===================>]   4.70M  25.6MB/s    in 0.2s    

2025-07-01 16:18:13 (25.6 MB/s) - ‘ml-100k.zip.2’ saved [4924029/4924029]

replace ml-100k/allbut.pl? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
import pandas as pd
from surprise import Dataset, Reader

# Load ratings into DataFrame
ratings_df = pd.read_csv('ml-100k/u.data', sep='\t', names=['userID', 'itemID', 'rating', 'timestamp'])

# Prepare data for Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_df[['userID', 'itemID', 'rating']], reader)


In [6]:
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy

# Split data
trainset, testset = train_test_split(data, test_size=0.2)

# Build and train model
algo = SVD()
algo.fit(trainset)
# Predict ratings
predictions = algo.test(testset)

# Evaluate using RMSE and MAE
print("Evaluation Metrics:")
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)
from collections import defaultdict

def get_top_n(predictions, n=5):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

top_n = get_top_n(predictions, n=5)
# Show recommendations for one user
for uid, user_ratings in top_n.items():
    print(f"\nTop recommendations for User {uid}:")
    for iid, rating in user_ratings:
        print(f"  Movie ID: {iid}, Predicted Rating: {rating:.2f}")
    break  # Remove break to show for all users


Evaluation Metrics:
RMSE: 0.9372
MAE:  0.7395

Top recommendations for User 878:
  Movie ID: 59, Predicted Rating: 3.91
  Movie ID: 22, Predicted Rating: 3.82
  Movie ID: 498, Predicted Rating: 3.70
  Movie ID: 427, Predicted Rating: 3.66
  Movie ID: 474, Predicted Rating: 3.63
